In [13]:
# 결재의견 찾는 로직 강화. (lxml 파서를 사용해야 <br/> 태그 이후의 <div>를 제대로 인식)

"""
 [기능]
 HTML 파일의 "결재의견" 테이블에서 의견을 추출하여
 DB documents 테이블의 activities JSON 내 actionComment 필드 업데이트

 [파서 선택]
 - lxml 파서 사용 (필수!)
 - html.parser는 <br/> 태그 이후의 <div>를 제대로 인식 못함

 [HTML 파싱 로직]
 1. <table summary='결재의견'> 테이블 찾기
 2. span.user에서 이름(F_12_black_b), 시간(F_11_gray) 추출
 3. user_span의 next_siblings 순회하여 의견 추출
    - <br> 태그 이후의 <div> 또는 텍스트 노드에서 추출
    - 다음 user_span 만나면 중단

 [DB 업데이트 로직]
 1. 특정 end_year 문서의 activities 조회
 2. 모든 actionComment를 빈값으로 초기화
 3. HTML에서 추출한 의견과 매칭 (name + actionDate 동시 일치)
 4. 매칭 성공 시 actionComment 업데이트
 5. 변경된 activities JSON을 DB에 UPDATE

 [시간대 처리]
 - HTML의 시간을 KST로 파싱 → Unix timestamp(ms)로 변환
 - DB의 actionDate와 정확히 매칭

 [디버깅 기능]
 - 특정 source_id(예: 2002153) 처리 시 상세 로그 출력
 - siblings 탐색 과정, 매칭 과정 추적

 [설정 (#여기를 수정하세요)]
 - base_path: HTML 폴더 경로
 - end_year: 대상 연도 (WHERE 조건)
 - db_config: DB 접속 정보

 [의존성]
 - beautifulsoup4
 - lxml (파서)
 - pymysql
 - pytz
  -> pip install beautifulsoup4 lxml pymysql pytz
"""

import os
import json
import re
from datetime import datetime
import pytz
from bs4 import BeautifulSoup
import pymysql
from pathlib import Path

class ActionCommentUpdater:
    def __init__(self, base_path):
        self.base_path = base_path
        self.kst = pytz.timezone('Asia/Seoul')
        
    def extract_source_id(self, filename):
        """파일명에서 마지막 숫자 추출"""
        numbers = re.findall(r'\d+', filename)
        return numbers[-1] if numbers else None
    
    def extract_person_info(self, text):
        """이름/직책/부서 형식에서 이름만 추출"""
        text = re.sub(r'\d+', '', text).strip()
        parts = text.split('/')
        if len(parts) >= 1:
            return parts[0].strip()
        return None
    
    def parse_html_for_comments(self, html_path):
        """HTML 파일에서 source_id와 결재의견 추출"""
        with open(html_path, 'r', encoding='utf-8') as f:
            # lxml 파서 사용 (html.parser는 일부 HTML을 제대로 파싱 못함)
            soup = BeautifulSoup(f.read(), 'lxml')
        
        filename = os.path.basename(html_path)
        source_id = self.extract_source_id(filename)
        
        # 결재의견 영역 찾기
        activities = []
        
        # "결재의견" 테이블 찾기
        approval_table = soup.find('table', summary='결재의견')
        if not approval_table:
            return {'sourceId': source_id, 'activities': activities}
        
        # td 안의 모든 span.user 찾기
        user_spans = approval_table.find_all('span', class_='user')
        
        for user_span in user_spans:
            # 이름 추출
            name_elem = user_span.find('span', class_='F_12_black_b')
            if not name_elem:
                continue
            
            name = self.extract_person_info(name_elem.get_text(strip=True))
            if not name:
                continue
            
            # 시간 추출
            time_elem = user_span.find('span', class_='F_11_gray')
            action_date = None
            if time_elem:
                time_text = time_elem.get_text(strip=True)
                try:
                    dt_naive = datetime.strptime(time_text, '%Y-%m-%d %H:%M:%S')
                    dt_kst = self.kst.localize(dt_naive)
                    action_date = int(dt_kst.timestamp() * 1000)
                except:
                    pass
            
            # 의견 추출: user_span의 다음 형제들에서 div 또는 텍스트 찾기
            action_comment = ""
            
            # 디버깅: 2002153인 경우 상세 로그
            if source_id == '2002153':
                print(f"\n  [{name}] siblings 탐색:")
                # 먼저 모든 siblings를 확인
                all_siblings = list(user_span.next_siblings)
                for idx, s in enumerate(all_siblings[:10]):  # 최대 10개까지
                    s_type = type(s).__name__
                    s_name = getattr(s, 'name', 'N/A')
                    s_class = s.get('class', []) if hasattr(s, 'get') else []
                    s_text = str(s)[:50] if s_type == 'NavigableString' else (s.get_text(strip=True)[:30] if hasattr(s, 'get_text') else '')
                    print(f"    sibling[{idx}]: type={s_type}, name={s_name}, class={s_class}, text={repr(s_text)}")
            
            # user_span의 next_siblings에서 div 또는 의견 텍스트 찾기
            br_found = False
            for idx, sibling in enumerate(user_span.next_siblings):
                if hasattr(sibling, 'name') and sibling.name:
                    # br 태그를 만나면 플래그 설정
                    if sibling.name == 'br':
                        br_found = True
                        continue
                    
                    # 다음 user_span을 만나면 중단
                    if sibling.name == 'span' and 'user' in sibling.get('class', []):
                        if source_id == '2002153':
                            print(f"    → 다음 user_span 발견, 중단")
                        break
                    
                    # div를 찾으면 의견 추출
                    if sibling.name == 'div':
                        action_comment = sibling.get_text(strip=True)
                        if source_id == '2002153':
                            print(f"    → div 발견! 의견: {action_comment[:50]}")
                        break
                else:
                    # NavigableString (텍스트 노드)인 경우
                    # br 다음에 오는 의미있는 텍스트를 추출
                    if br_found and sibling and str(sibling).strip():
                        action_comment = str(sibling).strip()
                        if source_id == '2002153':
                            print(f"    → 텍스트 노드 발견! 의견: {action_comment[:50]}")
                        break
            
            activities.append({
                'name': name,
                'actionDate': action_date,
                'actionComment': action_comment
            })
        
        return {
            'sourceId': source_id,
            'activities': activities
        }

        
    def process_all_html_files(self):
        """모든 HTML 파일에서 결재의견 추출"""
        all_results = {}
        approval_path = Path(self.base_path) / '결재'
        
        if not approval_path.exists():
            print(f"경로를 찾을 수 없습니다: {approval_path}")
            return all_results
            
        html_files = list(approval_path.rglob('*.html'))
        print(f"총 {len(html_files)}개의 HTML 파일을 찾았습니다.")
        
        for idx, html_file in enumerate(html_files, 1):
            try:
                if idx % 100 == 0 or idx == len(html_files):
                    print(f"HTML 파싱 중... [{idx}/{len(html_files)}] {html_file.name}")
                result = self.parse_html_for_comments(html_file)
                source_id = result['sourceId']
                if source_id:
                    all_results[source_id] = result['activities']
                    
                    # 디버깅: 2002153 파일 확인
                    if source_id == '2002153':
                        print(f"\n★★★ HTML 파싱 완료: source_id=2002153 ★★★")
                        print(f"파일명: {html_file.name}")
                        print(f"activities 개수: {len(result['activities'])}")
                        
            except Exception as e:
                print(f"오류 발생 ({html_file.name}): {e}")
                import traceback
                if source_id == '2002153':
                    traceback.print_exc()
        
        print(f"\n✅ HTML 파싱 완료: {len(all_results)}개 문서\n")
        
        # 최종 확인
        if '2002153' in all_results:
            print(f"★ 2002153 최종 확인:")
            for idx, act in enumerate(all_results['2002153']):
                print(f"  [{idx}] {act['name']}, {act['actionDate']}, 의견: {act['actionComment'][:30] if act['actionComment'] else '(없음)'}")
        
        return all_results
    
    def update_action_comments(self, db_config, end_year, html_comments):
        """DB의 결재의견 업데이트"""
        conn = None
        cursor = None
        
        try:
            print("=== DB 연결 시작 ===")
            conn = pymysql.connect(
                host=db_config['host'],
                user=db_config['user'],
                password=db_config['password'],
                database=db_config['database'],
                charset='utf8mb4'
            )
            print("✓ DB 연결 성공\n")
            cursor = conn.cursor(pymysql.cursors.DictCursor)
            
            # 디버깅: HTML에 2002153 있는지 확인
            if '2002153' in html_comments:
                print(f"★★★ HTML 딕셔너리에 2002153 존재 확인! ★★★")
                print(f"HTML activities 개수: {len(html_comments['2002153'])}")
                for idx, act in enumerate(html_comments['2002153']):
                    print(f"  [{idx}] 이름:{act['name']}, 날짜:{act['actionDate']}, 의견:{act['actionComment'][:50] if act['actionComment'] else '(없음)'}")
            else:
                print(f"⚠️ HTML 딕셔너리에 2002153 없음!")
            print()
            
            # Step 1: end_year = 2010인 documents 조회
            print(f"=== Step 1: end_year={end_year} 문서 조회 ===")
            cursor.execute("""
                SELECT id, source_id, activities
                FROM documents
                WHERE end_year = %s
            """, (end_year,))
            
            documents = cursor.fetchall()
            print(f"✓ 조회 완료: {len(documents)}건\n")
            
            # Step 2 & 3: 결재의견 초기화 및 업데이트
            print(f"=== Step 2 & 3: 결재의견 초기화 및 업데이트 ===")
            
            total_count = 0
            updated_count = 0
            matched_count = 0
            error_count = 0
            
            for doc in documents:
                total_count += 1
                
                if total_count % 1000 == 0:
                    print(f"처리 중... {total_count}/{len(documents)}건 (매칭: {matched_count}건)")
                
                doc_id = doc['id']
                source_id = doc['source_id']
                activities_str = doc['activities']
                
                if not activities_str:
                    continue
                
                try:
                    # activities JSON 파싱
                    activities = json.loads(activities_str)
                    
                    # 디버깅: 2002153 처리 시작
                    if source_id == '2002153':
                        print(f"\n★★★ DB 처리: source_id=2002153 ★★★")
                        print(f"DB id: {doc_id}")
                        print(f"DB activities 개수: {len(activities)}")
                        for idx, act in enumerate(activities):
                            print(f"  DB[{idx}] 이름:{act['name']}, 날짜:{act['actionDate']}, 의견:{act.get('actionComment', '(없음)')[:30]}")
                    
                    # 모든 actionComment 초기화
                    for activity in activities:
                        activity['actionComment'] = ''
                    
                    # HTML에서 추출한 결재의견이 있으면 매칭
                    if source_id in html_comments:
                        html_activities = html_comments[source_id]
                        
                        # 디버깅: 2002153 매칭 시작
                        if source_id == '2002153':
                            print(f"\n  → 매칭 시작...")
                            print(f"  DB activities:")
                            for idx, act in enumerate(activities):
                                from datetime import datetime
                                if act.get('actionDate'):
                                    dt_str = datetime.fromtimestamp(act['actionDate']/1000).strftime('%Y-%m-%d %H:%M:%S')
                                else:
                                    dt_str = 'None'
                                print(f"    DB[{idx}] {act['name']} / {act['actionDate']} ({dt_str})")
                            print(f"  HTML activities:")
                            for idx, act in enumerate(html_activities):
                                if act.get('actionDate'):
                                    dt_str = datetime.fromtimestamp(act['actionDate']/1000).strftime('%Y-%m-%d %H:%M:%S')
                                else:
                                    dt_str = 'None'
                                print(f"    HTML[{idx}] {act['name']} / {act['actionDate']} ({dt_str}) / {act['actionComment'][:30] if act['actionComment'] else '(없음)'}")
                        
                        # name + actionDate 매칭
                        for db_activity in activities:
                            db_name = db_activity.get('name', '')
                            db_date = db_activity.get('actionDate')
                            
                            for html_activity in html_activities:
                                html_name = html_activity.get('name', '')
                                html_date = html_activity.get('actionDate')
                                html_comment = html_activity.get('actionComment', '')
                                
                                # 이름과 날짜가 모두 일치
                                if db_name == html_name and db_date == html_date:
                                    db_activity['actionComment'] = html_comment
                                    if html_comment:  # 빈값이 아닌 의견만 카운트
                                        matched_count += 1
                                    
                                    # 디버깅: 2002153 매칭 성공
                                    if source_id == '2002153':
                                        print(f"  ✓ 매칭 성공: {db_name}, {db_date}")
                                        print(f"    의견: {html_comment[:50] if html_comment else '(없음)'}")
                                    break
                        
                        # 디버깅: 2002153 매칭 후 결과
                        if source_id == '2002153':
                            print(f"\n  → 매칭 후 DB activities:")
                            for idx, act in enumerate(activities):
                                print(f"  [{idx}] 의견:{act['actionComment'][:50] if act['actionComment'] else '(빈값)'}")
                    
                    # DB 업데이트
                    updated_activities_str = json.dumps(activities, ensure_ascii=False, separators=(',', ':'))
                    cursor.execute("""
                        UPDATE documents 
                        SET activities = %s 
                        WHERE id = %s
                    """, (updated_activities_str, doc_id))
                    conn.commit()
                    updated_count += 1
                    
                except json.JSONDecodeError as e:
                    error_count += 1
                    print(f"⚠️ JSON 파싱 에러 (id={doc_id}, source_id={source_id}): {str(e)}")
                except Exception as e:
                    error_count += 1
                    print(f"⚠️ 업데이트 에러 (id={doc_id}): {str(e)}")
                    conn.rollback()
            
            print(f"\n=== 결과 요약 ===")
            print(f"총 처리: {total_count}건")
            print(f"업데이트: {updated_count}건")
            print(f"의견 매칭: {matched_count}건 (빈값 제외)")
            if error_count > 0:
                print(f"⚠️ 에러: {error_count}건")
            
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            if conn:
                conn.rollback()
        
        finally:
            if cursor:
                cursor.close()
            if conn:
                conn.close()
                print("\n✓ DB 연결 종료")


def main():
    #여기를 수정하세요
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html'
    end_year = 2010
    
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval'
    }
    
    # 실행
    updater = ActionCommentUpdater(base_path)
    
    # 1. HTML에서 결재의견 추출
    print("="*60)
    html_comments = updater.process_all_html_files()
    
    # 2. DB 업데이트
    print("="*60)
    updater.update_action_comments(db_config, end_year, html_comments)
    print("="*60)
    
    print("\n✅ 모든 작업 완료!")


if __name__ == '__main__':
    main()

총 840개의 HTML 파일을 찾았습니다.

  [김하응] siblings 탐색:
    sibling[0]: type=NavigableString, name=None, class=[], text='\n'
    sibling[1]: type=Tag, name=br, class=[], text=''
    sibling[2]: type=NavigableString, name=None, class=[], text='\n'
    sibling[3]: type=Tag, name=div, class=[], text='처리부탁드립니다'
    sibling[4]: type=NavigableString, name=None, class=[], text='\n'
    sibling[5]: type=Tag, name=br, class=[], text=''
    sibling[6]: type=NavigableString, name=None, class=[], text='\n'
    sibling[7]: type=Tag, name=span, class=['user'], text='기안고화석/이사/(주)애니파이브2010-02-22 10'
    sibling[8]: type=NavigableString, name=None, class=[], text='\n'
    sibling[9]: type=Tag, name=br, class=[], text=''
    → div 발견! 의견: 처리부탁드립니다

  [고화석] siblings 탐색:
    sibling[0]: type=NavigableString, name=None, class=[], text='\n'
    sibling[1]: type=Tag, name=br, class=[], text=''
    sibling[2]: type=NavigableString, name=None, class=[], text='\n'
    sibling[3]: type=Tag, name=div, class=[], text='승인합니다

In [1]:
import os
import json
import re
from datetime import datetime
import pytz
from bs4 import BeautifulSoup
import pymysql
from pathlib import Path

class ActionCommentUpdater:
    def __init__(self, base_path):
        self.base_path = base_path
        self.kst = pytz.timezone('Asia/Seoul')
        
    def extract_source_id(self, filename):
        """파일명에서 마지막 숫자 추출"""
        numbers = re.findall(r'\d+', filename)
        return numbers[-1] if numbers else None
    
    def extract_person_info(self, text):
        """이름/직책/부서 형식에서 이름만 추출"""
        text = re.sub(r'\d+', '', text).strip()
        parts = text.split('/')
        if len(parts) >= 1:
            return parts[0].strip()
        return None
    
    def parse_html_for_comments(self, html_path):
        """HTML 파일에서 source_id와 결재의견 추출"""
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        
        filename = os.path.basename(html_path)
        source_id = self.extract_source_id(filename)
        
        # span.user에서 이름, 시간, 의견 추출
        activities = []
        user_spans = soup.find_all('span', class_='user')
        
        for user_span in user_spans:
            # 이름 추출
            name_elem = user_span.find('span', class_='F_12_black_b')
            if not name_elem:
                continue
            
            name = self.extract_person_info(name_elem.get_text(strip=True))
            if not name:
                continue
            
            # 시간 추출
            time_elem = user_span.find('span', class_='F_11_gray')
            action_date = None
            if time_elem:
                time_text = time_elem.get_text(strip=True)
                try:
                    # 2010-12-31 13:57:02 형식 파싱
                    dt_naive = datetime.strptime(time_text, '%Y-%m-%d %H:%M:%S')
                    dt_kst = self.kst.localize(dt_naive)
                    action_date = int(dt_kst.timestamp() * 1000)
                except:
                    pass
            
            # 의견 추출
            action_comment = ""
            for sibling in user_span.next_siblings:
                if hasattr(sibling, 'name'):
                    if sibling.name == 'span' and 'user' in sibling.get('class', []):
                        break
                    if sibling.name == 'div':
                        action_comment = sibling.get_text(strip=True)
                        break
            
            activities.append({
                'name': name,
                'actionDate': action_date,
                'actionComment': action_comment
            })
        
        return {
            'sourceId': source_id,
            'activities': activities
        }
    
    def process_all_html_files(self):
        """모든 HTML 파일에서 결재의견 추출"""
        all_results = {}
        approval_path = Path(self.base_path) / '결재'
        
        if not approval_path.exists():
            print(f"경로를 찾을 수 없습니다: {approval_path}")
            return all_results
            
        html_files = list(approval_path.rglob('*.html'))
        print(f"총 {len(html_files)}개의 HTML 파일을 찾았습니다.")
        
        for idx, html_file in enumerate(html_files, 1):
            try:
                if idx % 100 == 0 or idx == len(html_files):
                    print(f"HTML 파싱 중... [{idx}/{len(html_files)}] {html_file.name}")
                result = self.parse_html_for_comments(html_file)
                source_id = result['sourceId']
                if source_id:
                    all_results[source_id] = result['activities']
            except Exception as e:
                print(f"오류 발생 ({html_file.name}): {e}")
        
        print(f"\n✅ HTML 파싱 완료: {len(all_results)}개 문서\n")
        return all_results
    
    def update_action_comments(self, db_config, end_year, html_comments):
        """DB의 결재의견 업데이트"""
        conn = None
        cursor = None
        
        try:
            print("=== DB 연결 시작 ===")
            conn = pymysql.connect(
                host=db_config['host'],
                user=db_config['user'],
                password=db_config['password'],
                database=db_config['database'],
                charset='utf8mb4'
            )
            print("✓ DB 연결 성공\n")
            cursor = conn.cursor(pymysql.cursors.DictCursor)
            
            # Step 1: end_year = 2010인 documents 조회
            print(f"=== Step 1: end_year={end_year} 문서 조회 ===")
            cursor.execute("""
                SELECT id, source_id, activities
                FROM documents
                WHERE end_year = %s
            """, (end_year,))
            
            documents = cursor.fetchall()
            print(f"✓ 조회 완료: {len(documents)}건\n")
            
            # Step 2 & 3: 결재의견 초기화 및 업데이트
            print(f"=== Step 2 & 3: 결재의견 초기화 및 업데이트 ===")
            
            total_count = 0
            updated_count = 0
            matched_count = 0
            error_count = 0
            
            for doc in documents:
                total_count += 1
                
                if total_count % 1000 == 0:
                    print(f"처리 중... {total_count}/{len(documents)}건 (매칭: {matched_count}건)")
                
                doc_id = doc['id']
                source_id = doc['source_id']
                activities_str = doc['activities']
                
                if not activities_str:
                    continue
                
                try:
                    # activities JSON 파싱
                    activities = json.loads(activities_str)
                    
                    # 모든 actionComment 초기화
                    for activity in activities:
                        activity['actionComment'] = ''
                    
                    # HTML에서 추출한 결재의견이 있으면 매칭
                    if source_id in html_comments:
                        html_activities = html_comments[source_id]
                        
                        # name + actionDate 매칭
                        for db_activity in activities:
                            db_name = db_activity.get('name', '')
                            db_date = db_activity.get('actionDate')
                            
                            for html_activity in html_activities:
                                html_name = html_activity.get('name', '')
                                html_date = html_activity.get('actionDate')
                                html_comment = html_activity.get('actionComment', '')
                                
                                # 이름과 날짜가 모두 일치
                                if db_name == html_name and db_date == html_date:
                                    db_activity['actionComment'] = html_comment
                                    if html_comment:  # 빈값이 아닌 의견만 카운트
                                        matched_count += 1
                                    break
                    
                    # DB 업데이트
                    updated_activities_str = json.dumps(activities, ensure_ascii=False, separators=(',', ':'))
                    cursor.execute("""
                        UPDATE documents 
                        SET activities = %s 
                        WHERE id = %s
                    """, (updated_activities_str, doc_id))
                    conn.commit()
                    updated_count += 1
                    
                except json.JSONDecodeError as e:
                    error_count += 1
                    print(f"⚠️ JSON 파싱 에러 (id={doc_id}, source_id={source_id}): {str(e)}")
                except Exception as e:
                    error_count += 1
                    print(f"⚠️ 업데이트 에러 (id={doc_id}): {str(e)}")
                    conn.rollback()
            
            print(f"\n=== 결과 요약 ===")
            print(f"총 처리: {total_count}건")
            print(f"업데이트: {updated_count}건")
            print(f"의견 매칭: {matched_count}건 (빈값 제외)")
            if error_count > 0:
                print(f"⚠️ 에러: {error_count}건")
            
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            if conn:
                conn.rollback()
        
        finally:
            if cursor:
                cursor.close()
            if conn:
                conn.close()
                print("\n✓ DB 연결 종료")


def main():
    # 설정
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html'
    end_year = 2010
    
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval'
    }
    
    # 실행
    updater = ActionCommentUpdater(base_path)
    
    # 1. HTML에서 결재의견 추출
    print("="*60)
    html_comments = updater.process_all_html_files()
    
    # 2. DB 업데이트
    print("="*60)
    updater.update_action_comments(db_config, end_year, html_comments)
    print("="*60)
    
    print("\n✅ 모든 작업 완료!")


if __name__ == '__main__':
    main()

총 840개의 HTML 파일을 찾았습니다.
HTML 파싱 중... [100/840] 20100225_연차휴가(3_2) 신청합니다._2002163.html
HTML 파싱 중... [200/840] 20100401_[기술지원팀 격려 인센티브 지급품의]_2002268.html
HTML 파싱 중... [300/840] 20100517_개인휴가품의_2002367.html
HTML 파싱 중... [400/840] 20100705_파일서버 접근 권한 요청_2002467.html
HTML 파싱 중... [500/840] 20100817_8월 아웃소싱 인건비 지급 품의-기술지원부문_2002570.html
HTML 파싱 중... [600/840] 20100929_재직증명서 발급요청_2002666.html
HTML 파싱 중... [700/840] 20101110_대전지사 공용 하드디스크 구매 요청_2002767.html
HTML 파싱 중... [800/840] 20101221_12월 아웃소싱 인건비 지급 품의-전문가Gr.(김규일)_2002865.html
HTML 파싱 중... [840/840] 20101231_법인카드 사용내역 정산 보고(10월,11월,12월)_2002907.html

✅ HTML 파싱 완료: 840개 문서

=== DB 연결 시작 ===
✓ DB 연결 성공

=== Step 1: end_year=2010 문서 조회 ===
✓ 조회 완료: 840건

=== Step 2 & 3: 결재의견 초기화 및 업데이트 ===

=== 결과 요약 ===
총 처리: 840건
업데이트: 840건
의견 매칭: 361건 (빈값 제외)

✓ DB 연결 종료

✅ 모든 작업 완료!
